### DESCRIPTION:
    This example shows how to generate a query json from user input using OpenAI GPT3.5 completion model
### REQUIREMENTS:
    Create an .env file with your OpenAI API key and save it in the root directory of this project with the following
    OPENAI_DEPLOYMENT_ENDPOINT ="<your openai endpoint>" 
    OPENAI_API_KEY = "<your openai api key>"
    OPENAI_DEPLOYMENT_NAME = "<your gpt35 deployment name>"
    OPENAI_DEPLOYMENT_VERSION = "<gpt35 api version>"
    OPENAI_MODEL_NAME="<gpt35 model name>"


In [2]:
from dotenv import load_dotenv
import pandas as pd
import utils
import os
import openai
import json 

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")
# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [3]:
template_prefix = """
<|im_start|>system
You are an assistant designed to extract entities from a user question. Users will paste in a string of text and you will respond with entities you have extracted from the text as a JSON object. Here is an example of your output format:
{duration:\"\",mood:[],instrument:[],genre:[], suggestions:mood:[],instrument:[],genre:[]}  
"""
template_sufix = "<|im_end|>\n<|im_start|>assistant"

In [4]:
def pretty_print_json_string(json_string):
    json_object = json.loads(json_string)
    json_formatted_str = json.dumps(json_object, indent=2)
    print(json_formatted_str)

def call_openai(text):
    prompt = template_prefix + text + template_sufix
    response = openai.Completion.create(
        engine=utils.OPENAI_DEPLOYMENT_NAME,
        prompt=prompt,
        temperature=0,
        max_tokens=4096,
        top_p=0.95,
        frequency_penalty=0,
        presence_penalty=0,
        stop=["<|im_end|>"])
    response = response['choices'][0]['text']
    response = utils.remove_chars("\n", response)
    response=utils.start_after_string("Answer:", response)
    response=utils.remove_tail_tags("<|im_end|>", response)
    print("Order: \n")
    pretty_print_json_string(response)

In [5]:
call_openai("I would like to find music for cello or piano, for a romantic mood")

Order: 

{
  "duration": "",
  "mood": [
    "romantic"
  ],
  "instrument": [
    "cello",
    "piano"
  ],
  "genre": [],
  "suggestions": {
    "mood": [
      "classical",
      "jazz"
    ],
    "instrument": [
      "violin",
      "guitar"
    ],
    "genre": [
      "romantic",
      "contemporary"
    ]
  }
}
